In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import glob
import os

import librosa.display
import soundfile

# Wrap related functions into a file: src/feature_extraction.py

```python
# Classes in the dataset
activities ={
  '1_uncap_':1,
  '2_click_':2,
  '3_exhale_out_':3,
  '4_inhale_':4,
  '5_exhale_into_':5,
}
```

# 1. Have a look at the feature dimentions

In [2]:
from src.feature_extraction import *

In [3]:
# load a sample
with soundfile.SoundFile( r'..\0. Data\4. Segments_5_classes\2_click_6.wav') as audio:
    waveform = audio.read(dtype='float32')
    sample_rate = audio.samplerate

In [4]:
spectrogram = feature_spectrogram(waveform, sample_rate)
spectrogram_mean = feature_spectrogram_mean(waveform, sample_rate)
centroid = feature_centroid(waveform, sample_rate)
bandwidth = feature_bandwidth(waveform, sample_rate)
# upper_bound = centroid[0] + bandwidth[0]
# lower_bound = centroid[0] - bandwidth[0]
melspectrogram = feature_melspectrogram(waveform, sample_rate)
melspectrogram_mean = feature_melspectrogram_mean(waveform, sample_rate)
mfcc = feature_mfcc(waveform, sample_rate)
chromagram = feature_chromagram(waveform, sample_rate)
chromagram_mean = feature_chromagram_mean(waveform, sample_rate)

C:\Users\ziyif\miniconda3\envs\sklearn\lib\site-packages\librosa\feature\spectral.py:2143: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  mel_basis = filters.mel(sr=sr, n_fft=n_fft, **kwargs)


In [5]:
spectrogram.shape

(1025, 63)

In [6]:
spectrogram_mean.shape

(1025,)

In [7]:
centroid.shape

(1, 63)

In [8]:
bandwidth.shape

(1, 63)

In [9]:
melspectrogram.shape

(128, 63)

In [10]:
melspectrogram_mean.shape

(128,)

In [11]:
mfcc.shape

(40,)

In [12]:
chromagram.shape

(12, 63)

In [13]:
chromagram_mean.shape

(12,)

# 2. Scaling, splitting and save the features

Data Split & Feature Transformation

In [14]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

## 2.1 Data Split

In [16]:
## Extract activity
def load_data(feature_set_number):
    data_folder = r'..\0. Data\4. Segments_5_classes' 
    X, y = [], []
    count = 0

    # Activity mapping table (ensure suffix has "_" to avoid misidentification)
    activities = {
        '1_uncap_': 1,
        '2_click_': 2,
        '3_exhale_out_': 3,
        '4_inhale_': 4,
        '5_exhale_into_': 5,
    }

    for file in glob.glob(os.path.join(data_folder, "*.wav")):
        file_name = os.path.basename(file)

        # Automatically match activity prefix
        activity = None
        for prefix in activities:
            if file_name.startswith(prefix):
                activity = activities[prefix]
                break
        if activity is None:
            print(f"⚠️ Skipping unrecognized file: {file_name}")
            continue

        # Extract features
        if feature_set_number == 6:
            features = create_feature_set_6(file)
        else:
            raise ValueError("❌ Invalid feature_set_number. Must be 1-6.")

        X.append(features)
        y.append(activity)
        count += 1
        print('\r' + f'🔄 Processed {count} audio samples', end='')

    print("\n✅ Data loading complete.")
    return np.array(X), np.array(y)


## 2.1.6 Feature set 6

In [17]:
X,y = load_data(feature_set_number=6)

X_train_6, X_test_6, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True)

# Output the train and test data size and shape
print(f"Train and Test Size {len(X_train_6)}, {len(X_test_6)}")
print("Feature set shape: ", X_train_6.shape)

C:\Users\ziyif\miniconda3\envs\sklearn\lib\site-packages\librosa\feature\spectral.py:2143: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  mel_basis = filters.mel(sr=sr, n_fft=n_fft, **kwargs)


🔄 Processed 540 audio samples
✅ Data loading complete.
Train and Test Size 432, 108
Feature set shape:  (432, 1331)


## 2.2 Save all feature sets

In [18]:
y_train = pd.DataFrame(y_train)
y_test = pd.DataFrame(y_test)

X_train_6 = pd.DataFrame(X_train_6)
X_test_6 = pd.DataFrame(X_test_6)

In [19]:
# Save the data to Excel
data_path = r'..\Features'

# Check if the directory exists, if not, create it
if not os.path.exists(data_path):
    os.makedirs(data_path)

# Save to csv files
file_path = os.path.join(data_path, 'y_train.csv')
y_train.to_csv(file_path)
print(f"File: {file_path}, saved")

file_path = os.path.join(data_path, 'y_test.csv')
y_test.to_csv(file_path)
print(f"File: {file_path}, saved")

file_path = os.path.join(data_path, 'X_train_6.csv')
X_train_6.to_csv(file_path)
print(f"File: {file_path}, saved")

file_path = os.path.join(data_path, 'X_test_6.csv')
X_test_6.to_csv(file_path)
print(f"File: {file_path}, saved")

File: C:\Users\ziyif\OneDrive\文档\GitHub\ML-Acoustic-DPI-Adherence\Features\y_train.csv, saved
File: C:\Users\ziyif\OneDrive\文档\GitHub\ML-Acoustic-DPI-Adherence\Features\y_test.csv, saved
File: C:\Users\ziyif\OneDrive\文档\GitHub\ML-Acoustic-DPI-Adherence\Features\X_train_6.csv, saved
File: C:\Users\ziyif\OneDrive\文档\GitHub\ML-Acoustic-DPI-Adherence\Features\X_test_6.csv, saved


# Wrap datasets into a file: src/data_loader.py

In [1]:
from src.data_loader import load_data

# Load the data
(y_train, y_test, 
X_train_6, X_test_6)= load_data(transform=True)

In [2]:
y_test.shape

(108, 1)

In [3]:
y_test

,0
0,2
1,1
2,1
3,1
4,4
...,...
103,1
104,2
105,3
106,4
